In [3]:
%pip install datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 3.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.1/316.1 kB 3.1 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.0/396.0 kB 2.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.4/28.4 MB 7.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 146.7/146.7 kB 3.5 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.7/53.7 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.6/81.6 kB 3.1 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.6.1
    Uninstalling fsspec-2024.6.1:
      Successfully uninstalled fsspec-2024.6.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning

In [96]:
%pip install peft

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 1.8 MB/s eta 0:00:0000:0100:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [4]:
%pip install setuptools

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 931.1/931.1 kB 4.2 MB/s eta 0:00:00a 0:00:01

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [23]:
%pip install sacrebleu

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 594.4 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 1.6 MB/s eta 0:00:00 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 8.4 MB/s eta 0:00:0000:0100:01
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [28]:
import tqdm
import torch
# import wandb
import pandas as pd
from sacrebleu import corpus_bleu
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from peft import LoraConfig, get_peft_model, TaskType
from torch.utils.data import TensorDataset, DataLoader
from transformers import BlenderbotForConditionalGeneration, AutoTokenizer, AdamW
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [101]:
model= BlenderbotForConditionalGeneration.from_pretrained('facebook/blenderbot-400M-distill', max_length=128)
tokenizer = AutoTokenizer.from_pretrained('facebook/blenderbot-400M-distill')



In [102]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
device

device(type='cpu')

In [103]:
loraConfig = LoraConfig(
    r=64,                                # Adjust based on model requirements
    lora_alpha=32,                       # Adjust based on model requirements
    target_modules=[
        "model.encoder.layers",           # Encoder layers for question processing
        "model.encoder.self_attn.q_proj", # Attention projection for question
        "model.encoder.self_attn.v_proj", # Attention projection for question
        "model.decoder.layers",           # Decoder layers for follow-up question generation
        "model.decoder.self_attn.q_proj", # Attention projection for follow-up question
        "model.decoder.self_attn.v_proj", # Attention projection for follow-up question
    ],
    lora_dropout=0.05,                   # Dropout rate
    bias="none",                         # Bias setting
    task_type=TaskType.CAUSAL_LM          # Task type for causal language modeling
)


In [104]:
model.config


BlenderbotConfig {
  "_name_or_path": "facebook/blenderbot-400M-distill",
  "activation_dropout": 0.0,
  "activation_function": "gelu",
  "add_bias_logits": false,
  "add_final_layer_norm": true,
  "architectures": [
    "BlenderbotForConditionalGeneration"
  ],
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "classif_dropout": 0.0,
  "classifier_dropout": 0.0,
  "d_model": 1280,
  "decoder_attention_heads": 32,
  "decoder_ffn_dim": 5120,
  "decoder_layerdrop": 0.0,
  "decoder_layers": 12,
  "decoder_start_token_id": 1,
  "do_blenderbot_90_layernorm": true,
  "dropout": 0.1,
  "encoder_attention_heads": 32,
  "encoder_ffn_dim": 5120,
  "encoder_layerdrop": 0.0,
  "encoder_layers": 2,
  "encoder_no_repeat_ngram_size": 3,
  "eos_token_id": 2,
  "extra_layer_norm": false,
  "extra_pos_embeddings": 0,
  "force_bos_token_to_be_generated": false,
  "forced_eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2"
  },
 

In [105]:
for name, param in model.named_parameters():
    print(name)

model.shared.weight
model.encoder.embed_positions.weight
model.encoder.layers.0.self_attn.k_proj.weight
model.encoder.layers.0.self_attn.k_proj.bias
model.encoder.layers.0.self_attn.v_proj.weight
model.encoder.layers.0.self_attn.v_proj.bias
model.encoder.layers.0.self_attn.q_proj.weight
model.encoder.layers.0.self_attn.q_proj.bias
model.encoder.layers.0.self_attn.out_proj.weight
model.encoder.layers.0.self_attn.out_proj.bias
model.encoder.layers.0.self_attn_layer_norm.weight
model.encoder.layers.0.self_attn_layer_norm.bias
model.encoder.layers.0.fc1.weight
model.encoder.layers.0.fc1.bias
model.encoder.layers.0.fc2.weight
model.encoder.layers.0.fc2.bias
model.encoder.layers.0.final_layer_norm.weight
model.encoder.layers.0.final_layer_norm.bias
model.encoder.layers.1.self_attn.k_proj.weight
model.encoder.layers.1.self_attn.k_proj.bias
model.encoder.layers.1.self_attn.v_proj.weight
model.encoder.layers.1.self_attn.v_proj.bias
model.encoder.layers.1.self_attn.q_proj.weight
model.encoder.la

In [106]:
dataStufff=pd.read_csv('AngularQuestions.csv')

In [107]:
train,val=train_test_split(dataStufff,test_size=0.2,random_state=42)

In [108]:
def prep(data, max=128):
    inputs_ids, attention_masks, labels = [], [], []
    for response, follow_up_question in zip(data['response'], data['follow_up_question']):
        encode_res = tokenizer(
            str(response),
            padding='max_length',
            truncation=True,
            max_length=max,
            return_tensors='pt'
        )
        encode_qeus = tokenizer(
            str(follow_up_question),
            padding='max_length',
            truncation=True,
            max_length=max,
            return_tensors='pt'
        )
        try:
            inputs_ids.append(encode_res['input_ids'].squeeze())
            attention_masks.append(encode_res['attention_mask'].squeeze())
            labels.append(encode_qeus['input_ids'].squeeze())
        except IndexError as e:
            print(f"IndexError occurred: {e}")
    return inputs_ids, attention_masks, labels


In [109]:
traininp,trainmask,trainlabel=prep(train)
valinp,valmask,vallabel=prep(val)

In [110]:
traininp    =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in traininp], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
trainmask   =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainmask], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
trainlabel  =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainlabel], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valinp      =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valinp], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
valmask     =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in valmask], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
vallabel    =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in vallabel], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)

/var/folders/tf/lh15p_h5417f4nb9dqtm2fhh0000gq/T/ipykernel_44929/3975725461.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  traininp    =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in traininp], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
/var/folders/tf/lh15p_h5417f4nb9dqtm2fhh0000gq/T/ipykernel_44929/3975725461.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainmask   =   torch.nn.utils.rnn.pad_sequence([torch.tensor(seq, dtype=torch.long) for seq in trainmask], batch_first=True, padding_value=tokenizer.pad_token_id).to(device)
/var/folders/tf/lh15p_h5417f4nb9dqtm2fhh0000gq/T/ipykernel_44929/3975725461.py:3: UserWarni

In [111]:
print(trainmask.max(),traininp.min())

tensor(1) tensor(0)


In [112]:
print(f"Max train input ID : {traininp.max().item()}, Vocal size: {tokenizer.vocab_size}")

Max train input ID : 8003, Vocal size: 8008


In [113]:
trainDataset=TensorDataset(traininp,trainmask,trainlabel)
trainDataloader=DataLoader(trainDataset,batch_size=16,shuffle=True)


In [114]:
type(trainDataloader)

torch.utils.data.dataloader.DataLoader

In [115]:
valDataset=TensorDataset(valinp,valmask,vallabel)
valDataloader=DataLoader(valDataset,batch_size=16,shuffle=True)

In [116]:
batchSize = 16
learningRate = 2e-5
epochs = 450

In [117]:
optimizer = AdamW(model.parameters(), lr=learningRate)
criterion = torch.nn.CrossEntropyLoss()

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [118]:
def train(model, optimizer, criterion, trainDataloader):
    model.train()
    total_loss = 0
    correct = 0
    for batch_idx, (input_ids, attention_mask, labels) in enumerate(trainDataloader):
        optimizer.zero_grad()
        input_ids = input_ids.long().to(device)
        attention_mask = attention_mask.long().to(device)
        labels = labels.long().to(device)
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        predictions = torch.argmax(outputs.logits, dim=-1)
        correct += (predictions == labels).sum().item()
    return total_loss / len(trainDataloader), correct / len(trainDataset)

In [119]:
def validate(model, criterion, valDataloader):
    model.eval()
    total_loss = 0
    correct_sequences = 0
    all_preds, all_labels = [], []
    with torch.no_grad():
        for batch_idx, (input_ids, attention_mask, labels) in enumerate(valDataloader):
            input_ids = input_ids.long().to(device)
            attention_mask = attention_mask.long().to(device)
            labels = labels.long().to(device)
            input_ids[input_ids >= tokenizer.vocab_size] = tokenizer.pad_token_id
            labels[labels >= tokenizer.vocab_size] = tokenizer.pad_token_id
            
            outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            total_loss += loss.item()
            predictions = torch.argmax(outputs.logits, dim=-1)
            
            correct_sequences += (predictions == labels).all(dim=1).sum().item()
            all_preds.extend(predictions.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    accuracy = correct_sequences / len(valDataloader.dataset)
    flattened_preds = [pred for seq in all_preds for pred in seq]
    flattened_labels = [label for seq in all_labels for label in seq]
    precision = precision_score(flattened_labels, flattened_preds, average="weighted", zero_division=0)
    recall = recall_score(flattened_labels, flattened_preds, average="weighted", zero_division=0)
    decoded_preds = [tokenizer.decode(pred, skip_special_tokens=True) for pred in all_preds]
    decoded_labels = [[tokenizer.decode(label, skip_special_tokens=True)] for label in all_labels]

    bleu_score = corpus_bleu(decoded_preds, decoded_labels)

    return total_loss / len(valDataloader), accuracy, precision, recall, bleu_score

In [120]:
for epoch in tqdm.trange(epochs,desc='Epoch(s)'):
    train_loss,train_acc=train(model,optimizer,criterion,trainDataloader)
    val_loss,val_acc,val_prec,val_recall,bleu_score=validate(model,criterion,valDataloader)
    dataDict={
          "Epoch": epoch+1,
        "Train Loss": train_loss,
        "Train Accuracy": train_acc,
        "Val Loss": val_loss,
        "Val Accuracy": val_acc,
        "Val Precision": val_prec,
        "Val Recall": val_recall,
        "BLEU Score": bleu_score

    }
    print(dataDict)
 
    model.save_pretrained("tunedModels/trainedSmallBlenderBotLoRA - Epoch {}".format(epoch))

Epoch(s):   0%|          | 0/450 [00:00<?, ?it/s]